In [1]:
import numpy as np
print(np.__version__)
import tensorflow as tf
print(tf.__version__)

from tensorflow.python.client import device_lib

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

1.21.2
2.7.0
Num GPUs Available:  1


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4284301328641407535
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9896853504
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9512180525969563776
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:08:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [2]:
import os
import matplotlib.pyplot as plt

#set seed
from numpy.random import seed
seed(1)
#tf.random.set_seed(1745093)

#import libraries
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, LSTM, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import classification_report

In [3]:
#import normalized data
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')


In [4]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(Y_train))
print(np.shape(Y_test))

(565, 35, 75)
(143, 35, 75)
(565, 2)
(143, 2)


In [5]:
#check for duplicates
lista5 = []
count5 = 0
for i in range (len(X_train)):
    bambino9 = X_train[i]
    for j in range (len(X_test)):
        if i<j :
            bambino10 = X_test[j]
            flag = np.array_equal(bambino9, bambino10)
            if (flag == True) and (i not in lista5):
                count5+=1
                lista5.append(i)
print(count5)
print(lista5)

0
[]


In [6]:
#shuffling
from sklearn.utils import shuffle

X_train, Y_train = shuffle(X_train, Y_train)
X_test, Y_test = shuffle(X_test, Y_test)

In [7]:
#for conv2D
'''
# uncomment if needed without KNN
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_shape_sample = np.shape(X_train[0])
avg_len = len(X_train[0])
'''

'\n# uncomment if needed without KNN\nX_train = np.expand_dims(X_train, axis=-1)\nX_test = np.expand_dims(X_test, axis=-1)\nX_shape_sample = np.shape(X_train[0])\navg_len = len(X_train[0])\n'

In [8]:
X_shape_sample = np.shape(X_train[0])
avg_len = len(X_train[0])
              
print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_test))
print(np.shape(Y_test))
print(avg_len)

(565, 35, 75)
(565, 2)
(143, 35, 75)
(143, 2)
35


In [9]:
#KNN code
from sklearn import datasets
from sklearn.neighbors import kneighbors_graph
from sklearn.neighbors import KNeighborsClassifier
from math import sqrt
import random

#computing accuracy
def accur(y_test, y_test_pred):
    count = 0
    for i in range(np.shape(y_test)[0]):
        if y_test[i][0] == y_test_pred[i][0]:
            count+=1
    return count/np.shape(y_test)[0]*100

# FROM HERE 3NN
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train.reshape(-1,35*75), Y_train)
test_pred_3NN = (neigh.predict(X_test.reshape(-1,35*75)))
train_pred_3NN = (neigh.predict(X_train.reshape(-1,35*75)))
accuracy_test_3 = accur(Y_test, test_pred_3NN)
accuracy_train_3 = accur(Y_train,train_pred_3NN)
print('the accuracy on the test set of the 5NN is', accuracy_test_3)
print('the accuracy on the training set of the 5NN is', accuracy_train_3)


# FROM HERE 5NN
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train.reshape(-1,35*75), Y_train)
test_pred_5NN = (neigh.predict(X_test.reshape(-1,35*75)))
train_pred_5NN = (neigh.predict(X_train.reshape(-1,35*75)))
accuracy_test_5 = accur(Y_test, test_pred_5NN)
accuracy_train_5 = accur(Y_train,train_pred_5NN)
print('the accuracy on the test set of the 5NN is', accuracy_test_5)
print('the accuracy on the training set of the 5NN is', accuracy_train_5)




the accuracy on the test set of the 5NN is 94.4055944055944
the accuracy on the training set of the 5NN is 96.10619469026548
the accuracy on the test set of the 5NN is 90.9090909090909
the accuracy on the training set of the 5NN is 95.39823008849557


In [10]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import time

Y_train_SVM = np.argmax(Y_train,axis = -1)
Y_test_SVM = np.argmax(Y_test,axis = -1)

#SVM implementation
#param_grid={'C':[0.1,0.5,1,5],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf'], 'random_state':[5]}
param_grid={'C':[5],'gamma':[0.001],'kernel':['rbf']}
svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid)
start = time.time()
model.fit(X_train.reshape(-1,35*75),Y_train_SVM)
end = time.time()
print(model.best_params_)
print('time to train the SVM', end-start)

#accuracy
def accur(y_test, y_test_pred):
    count = 0
    for i in range(np.shape(y_test)[0]):
        if y_test[i] == y_test_pred[i]:
            count+=1
    return count/np.shape(y_test)[0]*100

y_pred_test_svm = model.predict(X_test.reshape(-1,35*75))
y_pred_train_svm = model.predict(X_train.reshape(-1,35*75))
print(f"The model is", accur(y_pred_test_svm,Y_test_SVM),'accurate')
print(f"The model is", accur(y_pred_train_svm,Y_train_SVM),'accurate on the training set')


{'C': 5, 'gamma': 0.001, 'kernel': 'rbf'}
time to train the SVM 1.4433116912841797
The model is 95.1048951048951 accurate
The model is 95.929203539823 accurate on the training set


In [11]:
import time
#grid search; to implement the actual grid search procedure add the values into the lists
units = [64] #i
n_filters = [58] #j
batch_size = [8] #k
seed = [1, 773, 999, 1234, 666,2,4,10,96,21] #s

best_param = [0,0,0]
max_seed = 0
acc_seed = []
risultati = []
ris = []

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_shape_sample = np.shape(X_train[0])
avg_len = len(X_train[0])

Y_test_b = np.argmax(Y_test, axis=-1) #back from to_categorical
Y_train_b = np.argmax(Y_train, axis = -1)

#computing accuracy
def accur(y_test, y_test_pred):
    count = 0
    for i in range(np.shape(y_test)[0]):
        if y_test[i] == y_test_pred[i]:
            count+=1
    return count/np.shape(y_test)[0]*100

train_acc_list = []
train_time_list = []
test_acc_list = []
def grid_search(n_filters, batch_size, units, best_param, seed, acc_seed, max_seed, ris, risultati):
    for i in range (len(units)): 
        for j in range (len(n_filters)):
            for k in range (len(batch_size)):

                for s in range (len(seed)):

                    #seed
                    tf.random.set_seed(seed[s])

                    #model
                    model = Sequential()
                    model.add(Conv2D(n_filters[j], (3,3), strides = (1,1), input_shape=X_shape_sample, padding='same', activation='relu', trainable = False)) #set trainable = True to fully train the network
                    model.add(Flatten())
                    model.add(Dense(units[i],activation='relu'))
                    model.add(Dropout(0.2))
                    model.add(Dense(2, activation = "softmax"))
                    model.summary()
                    early_stop = EarlyStopping(monitor='val_loss', patience=700, restore_best_weights = False)
                    opt =keras.optimizers.Adam(learning_rate=0.0001)
                    model.compile(loss='binary_crossentropy',optimizer=opt ,metrics=['accuracy'])
                    
                    start = time.time()
                    history = model.fit(X_train, Y_train, epochs=300, batch_size=batch_size[k], validation_split = 0.1 ,callbacks =[early_stop], verbose=0)
                    end = time.time()
                    print('time for training in seconds ',end-start)
                    train_time_list.append(end-start)
                    #prediction
                    y_predict = np.argmax(model.predict(X_test), axis=-1)
                    y_predict_train = np.argmax(model.predict(X_train),axis = -1)
                    accuracy = accur(Y_test_b, y_predict)
                    train_accuracy = accur(Y_train_b,y_predict_train)
                    train_acc_list.append(train_accuracy)
                    test_acc_list.append(accuracy)
                    
                    #plotting the confusion matrix
                    print('\ntraining confusion matrix \n ',tf.math.confusion_matrix(
                        Y_train_b, y_predict_train, num_classes=None, weights=None, dtype=tf.dtypes.int32,
                        name=None))

                    #plotting the confusion matrix
                    print('\nest confusion matrix \n ',tf.math.confusion_matrix(
                        Y_test_b, y_predict, num_classes=None, weights=None, dtype=tf.dtypes.int32,
                        name=None))
                    

                    #saving seed accuracy
                    acc_seed.append(accuracy)
                    print("seed", seed[s])
                    print("param", units[i], n_filters[j], batch_size[k])
                    print("acc", accuracy)

                #computing average accuracy 
                avg_seed = np.average(acc_seed)
                if avg_seed > max_seed:
                    max_seed = avg_seed
                    best_param[0] = units[i]
                    best_param[1]= n_filters[j]
                    best_param[2] = batch_size[k]

                #saving the results
                ris.append(avg_seed)
                ris.append(units[i])
                ris.append(n_filters[j])
                ris.append(batch_size[k])
                risultati.append(ris)
                ris = []
                print(risultati) 
                
                print("max_seed", max_seed)
                print("acc_seed", acc_seed)
                acc_seed = []
                avg_seed = 0

    return max_seed, best_param, risultati

grid = grid_search(n_filters, batch_size, units, best_param, seed, acc_seed, max_seed, ris, risultati)
print('average training time: ',sum(train_time_list)/len(seed))
print('average training accuracy: ',sum(train_acc_list)/len(seed))
print('average test accuracy: ',sum(test_acc_list)/len(seed))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 35, 75, 58)        580       
                                                                 
 flatten (Flatten)           (None, 152250)            0         
                                                                 
 dense (Dense)               (None, 64)                9744064   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 9,744,774
Trainable params: 9,744,774
Non-trainable params: 0
_________________________________________________________________
time for training in seconds  72.7042238712310

In [12]:
import statistics
std_training = statistics.stdev(train_acc_list)
std_test = statistics.stdev(test_acc_list)
print('std training = ',std_training, 'std_test = ',std_test)

std training =  0.5546977828220946 std_test =  0.9353101467477364
